In [1]:
'''
Reconstruction helical data using parallel conebeam rebinning
'''

'\nReconstruction helical data using parallel conebeam rebinning\n'

In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

import ct_projector.projector.numpy as ct_projector
import ct_projector.projector.numpy.helical_equiangular_parallel_rebin as ct_helical

In [3]:
input_filename = '54_1.mat'

with h5py.File(input_filename, 'r') as f:
    # if a view is valid
    view_valid_a = np.copy(f['sh']['Lookup']['DetA']).flatten()
    view_valid_b = np.copy(f['sh']['Lookup']['DetB']).flatten()
    
    # z position of the source for each view, convert to mm
    zpos_a = np.copy(f['posA']).flatten() / 1000
    zpos_b = np.copy(f['posB']).flatten() / 1000
    
    # angle of the source for each view, convert to radius
    angles_a = np.copy(f['angleA']).flatten() / 180 * np.pi
    angles_b = np.copy(f['angleB']).flatten() / 180 * np.pi
    
    # projection, convert to attenuation
    prjs_a = np.copy(np.copy(f['projA'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5
    prjs_b = np.copy(np.copy(f['projB'])[:, ::-1, :], 'C').astype(np.float32) / 2294.5

In [4]:
projector = ct_projector.ct_projector()
projector.nv = prjs_a.shape[1]
projector.dso = 595
projector.du = 0.067864004196156 * np.pi / 180 * projector.dsd
projector.dv = 1.0947
projector.off_u = -1.25
projector.rotview = 1152
dtheta = np.pi * 2 / projector.rotview

projector.nu = prjs_a.shape[2]
projector_rebin_a, prjs_rebin_a, angles_a, zrot_a, nview_margin_pre_a, nview_margin_post_a = \
    ct_helical.rebin_helical_to_parallel(projector, prjs_a, angles_a, zpos_a)

projector.nu = prjs_b.shape[2]
projector_rebin_b, prjs_rebin_b, angles_b, zrot_b, nview_margin_pre_b, nview_margin_post_b = \
    ct_helical.rebin_helical_to_parallel(projector, prjs_b, angles_b, zpos_b)

Beta (fan angle) interpolation...
100,200,300,400,500,600,700,
Theta (source angle) interpolation...
100,200,300,400,500,600,700,
Beta (fan angle) interpolation...
100,200,300,400,
Theta (source angle) interpolation...
100,200,300,400,
